In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, glob
import time

В начале указываем путь к набор данных

In [2]:
basepath = os.getcwd()
print(basepath)

str_path = str(basepath)
#str_path = str_path.replace("""\\""", """/""")

outcomes_path = str_path + "/Outcomes.txt"
print(outcomes_path)
str_path = str_path + '/set/'
print(str_path)


C:\Users\exebeche\Desktop\Classification\Reanimation
C:\Users\exebeche\Desktop\Classification\Reanimation/Outcomes.txt
C:\Users\exebeche\Desktop\Classification\Reanimation/set/


In [3]:
dir_names = os.listdir(str_path)
len(dir_names)

8000

In [4]:
df = pd.read_csv(str_path + dir_names[0], sep=",")
df

,Time,Parameter,Value
0,00:00,RecordID,132539.00
1,00:00,Age,54.00
2,00:00,Gender,0.00
3,00:00,Height,-1.00
4,00:00,ICUType,4.00
...,...,...,...
268,47:37,NIMAP,79.33
269,47:37,NISysABP,128.00
270,47:37,RespRate,23.00
271,47:37,Temp,37.80


Прежде чем составить датасет по всем файлам, пройдемся по каждому файлу, чтобы узнать уникальные имена признаков, они отличаются для некоторый файлов. Статические признаки одинаковые, для всех файлов

In [6]:
static_columns = ['RecordID','Age','Gender','Height','ICUType', 'Weight']

In [7]:
%%time
dynamic_columns = set()

for path_txt in dir_names:
    if os.path.isfile(str_path + path_txt):
        df = pd.read_csv(str_path + path_txt, sep=",")
        #в df считываются текстовые файлы
        a = df['Parameter']
        a = np.array(a)
        column_names = np.unique(a) 
        column_names = column_names.tolist()
                   
        #column_names список названий параметров 
        for i in range(0, len(column_names)):
            dynamic_columns.add(column_names[i])
            
for i in range(0, len(static_columns)):
    if static_columns[i] in column_names:
        dynamic_columns.remove(static_columns[i])

Wall time: 18.5 s


In [8]:
dynamic_columns

{'ALP',
 'ALT',
 'AST',
 'Albumin',
 'BUN',
 'Bilirubin',
 'Cholesterol',
 'Creatinine',
 'DiasABP',
 'FiO2',
 'GCS',
 'Glucose',
 'HCO3',
 'HCT',
 'HR',
 'K',
 'Lactate',
 'MAP',
 'MechVent',
 'Mg',
 'NIDiasABP',
 'NIMAP',
 'NISysABP',
 'Na',
 'PaCO2',
 'PaO2',
 'Platelets',
 'RespRate',
 'SaO2',
 'SysABP',
 'Temp',
 'TroponinI',
 'TroponinT',
 'Urine',
 'WBC',
 'pH'}

8000 файлов могут обрабатываются достаточно долго, поэтому имеет смымсл считывать набор данных по частям

Каждый файл будет считываться в строку, для каждого параметра будет браться средние значение, можно взять и другие статистические признаки, но это займёт больше времени.

Считывание организованно так, чтобы можно было считывать файлы частями, это сделанно, чтобы можно было оценить время на считывание и на случай, если данные не успеют считаться 2 признака 900 файлов 1min 56s

In [11]:
rows_ = []  # Список строк датасета - динамическая структрa, значит можно добавлять в нее новые данные постепенно

In [12]:
%%time

first_file_num = 0
last_file_num = 8000

for i in range(first_file_num, len(dir_names)):
    if(i == last_file_num):
        break
    if os.path.isfile(str_path + dir_names[i]):
        df = pd.read_csv(str_path + dir_names[i], sep=",")              
        data_row = pd.DataFrame({'RecordID': df[df['Parameter'] == 'RecordID']['Value']})
        
        for i in range(1, len(static_columns)):
            rslt_df = df[df['Parameter'] == static_columns[i]]
            values_ = rslt_df['Value'].values
            dat_ = pd.DataFrame({static_columns[i]: [values_[0]]})
            data_row= data_row.join(dat_)
            
        for val in dynamic_columns:
            rslt_df = df[df['Parameter'] == val]
            values_list = rslt_df['Value'].values
            
            _mean = np.nanmean(values_list)
            _std = np.nanstd(values_list)
            
            _quan25 = np.nanquantile(values_list, 0.25)
            _quan75 = np.nanquantile(values_list, 0.75)
            
            if values_list.size != 0:
                _min = min(values_list)
                _max = max(values_list)
            else:
                _min = np.nan
                _max = np.nan
                
            dat_mean = pd.DataFrame({val + '_mean': [_mean]})
            dat_std = pd.DataFrame({val + '_std': [_std]})
            
            dat_min = pd.DataFrame({val + '_min': [_min]})
            dat_max = pd.DataFrame({val + '_max': [_max]})
            dat_quan25  = pd.DataFrame({val + 'quan25': [_quan25]})
            dat_quan75  = pd.DataFrame({val + 'quan75': [_quan75]})
            
            data_row = data_row.join(dat_mean)
            data_row = data_row.join(dat_std)
            
            data_row = data_row.join(dat_min)
            data_row = data_row.join(dat_max)
            
            data_row = data_row.join(dat_quan25)
            data_row = data_row.join(dat_quan75)
            
        rows_.append(data_row)

<timed exec>:21: RuntimeWarning: Mean of empty slice
C:\Users\UserName\anaconda3\envs\tf_gpu\lib\site-packages\numpy\lib\nanfunctions.py:1372: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\UserName\anaconda3\envs\tf_gpu\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\UserName\anaconda3\envs\tf_gpu\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\UserName\anaconda3\envs\tf_gpu\lib\site-packages\numpy\core\fromnumeric.py:3723: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\UserName\anaconda3\envs\tf_gpu\lib\site-packages\numpy\core\_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrme

Wall time: 43min 7s


In [13]:
len(rows_)

8000

In [14]:
columns = rows_[0].columns # Создаём пустой датасет с именами строк
index_ = []
for i in range(0, len(rows_)):
    index_.append(i)
df = pd.DataFrame(columns=columns, index= index_)
df

,RecordID,Age,Gender,Height,ICUType,Weight,ALP_mean,ALP_std,ALP_min,ALP_max,...,Bilirubin_min,Bilirubin_max,Bilirubinquan25,Bilirubinquan75,Albumin_mean,Albumin_std,Albumin_min,Albumin_max,Albuminquan25,Albuminquan75
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
%%time
for i in range(len(rows_)):#перенос данных из списка в датафрейм
    df.loc[i] = rows_[i].iloc[0]

Wall time: 7.72 s


In [16]:
df.RecordID = df.RecordID.astype(int)
df_outcomes = pd.read_csv(outcomes_path, sep=",")
df_outcomes 

,RecordID,Survival,In-hospital_death
0,132539,-1,0
1,132540,-1,0
2,132541,-1,0
3,132543,575,0
4,132545,918,0
...,...,...,...
7995,152849,752,0
7996,152851,39,0
7997,152858,334,0
7998,152862,-1,0


In [17]:
df['Survival'] = df_outcomes['Survival'].values
df['In-hospital_death'] = df_outcomes['In-hospital_death'].values
df

,RecordID,Age,Gender,Height,ICUType,Weight,ALP_mean,ALP_std,ALP_min,ALP_max,...,Bilirubinquan25,Bilirubinquan75,Albumin_mean,Albumin_std,Albumin_min,Albumin_max,Albuminquan25,Albuminquan75,Survival,In-hospital_death
0,132539,54.0,0.0,-1.0,4.0,-1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,0
1,132540,76.0,1.0,175.3,2.0,76.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,0
2,132541,44.0,0.0,-1.0,3.0,56.7,116.0,11.0,105.0,127.0,...,2.85,2.95,2.5,0.2,2.3,2.7,2.4,2.6,-1,0
3,132543,68.0,1.0,180.3,3.0,84.6,105.0,0.0,105.0,105.0,...,0.2,0.2,4.4,0.0,4.4,4.4,4.4,4.4,575,0
4,132545,88.0,0.0,-1.0,3.0,-1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,3.3,0.0,3.3,3.3,3.3,3.3,918,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,152849,78.0,1.0,180.3,2.0,83.18,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,752,0
7996,152851,90.0,1.0,177.8,3.0,87.0,74.0,0.0,74.0,74.0,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,39,0
7997,152858,70.0,0.0,152.4,2.0,74.0,88.0,0.0,88.0,88.0,...,NaN,NaN,2.8,0.0,2.8,2.8,2.8,2.8,334,0
7998,152862,49.0,0.0,-1.0,3.0,57.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,0


In [18]:
df.columns

Index(['RecordID', 'Age', 'Gender', 'Height', 'ICUType', 'Weight', 'ALP_mean',
       'ALP_std', 'ALP_min', 'ALP_max',
       ...
       'Bilirubinquan25', 'Bilirubinquan75', 'Albumin_mean', 'Albumin_std',
       'Albumin_min', 'Albumin_max', 'Albuminquan25', 'Albuminquan75',
       'Survival', 'In-hospital_death'],
      dtype='object', length=224)

In [19]:
df.to_csv('results.csv', index=False)

# ВТОРАЯ ЧАСТЬ ПРЕДОБРАБОТКА

In [21]:
# Вывод всех столбцов, где есть Nan

isna_series = df.isna().sum() 
count = 0
for i in range(0, isna_series.size):
    if(isna_series[i] > 0):
        print(isna_series.index[i], '  ', isna_series[i])
        count+=1

print("Count = ", count)

ALP_mean    4621
ALP_std    4621
ALP_min    4621
ALP_max    4621
ALPquan25    4621
ALPquan75    4621
Platelets_mean    130
Platelets_std    130
Platelets_min    130
Platelets_max    130
Plateletsquan25    130
Plateletsquan75    130
RespRate_mean    5772
RespRate_std    5772
RespRate_min    5772
RespRate_max    5772
RespRatequan25    5772
RespRatequan75    5772
Cholesterol_mean    7352
Cholesterol_std    7352
Cholesterol_min    7352
Cholesterol_max    7352
Cholesterolquan25    7352
Cholesterolquan75    7352
SysABP_mean    2390
SysABP_std    2390
SysABP_min    2390
SysABP_max    2390
SysABPquan25    2390
SysABPquan75    2390
MAP_mean    2402
MAP_std    2402
MAP_min    2402
MAP_max    2402
MAPquan25    2402
MAPquan75    2402
AST_mean    4539
AST_std    4539
AST_min    4539
AST_max    4539
ASTquan25    4539
ASTquan75    4539
Lactate_mean    3602
Lactate_std    3602
Lactate_min    3602
Lactate_max    3602
Lactatequan25    3602
Lactatequan75    3602
NIDiasABP_mean    999
NIDiasABP_std    999

In [22]:
# если столбец содержит больше 30% пропущенных значений, то удаляем его
for i in range(0, isna_series.size):
    if(isna_series[i] > 2400):
        df = df.drop(isna_series.index[i], axis = 1)

In [23]:
df = df.fillna(df.mean(axis = 0))
df.head()

#df = df_nan.interpolate(method = 'linear').fillna(0)
#df.head()

,RecordID,Age,Gender,Height,ICUType,Weight,Platelets_mean,Platelets_std,Platelets_min,Platelets_max,...,BUNquan25,BUNquan75,WBC_mean,WBC_std,WBC_min,WBC_max,WBCquan25,WBCquan75,Survival,In-hospital_death
0,132539,54.0,0.0,-1.0,4.0,-1.0,203.000000,18.000000,185.0,221.0,...,9.25,11.75,10.300000,0.900000,9.4,11.2,9.85,10.75,-1,0
1,132540,76.0,1.0,175.3,2.0,76.0,178.600000,29.789931,135.0,226.0,...,17.00,19.50,11.266667,2.735365,7.4,13.3,10.25,13.20,-1,0
2,132541,44.0,0.0,-1.0,3.0,56.7,89.666667,17.211108,72.0,113.0,...,3.00,5.50,4.700000,1.080123,3.7,6.2,3.95,5.20,-1,0
3,132543,68.0,1.0,180.3,3.0,84.6,330.000000,44.951826,284.0,391.0,...,15.00,21.50,9.400000,1.529706,7.9,11.5,8.35,10.15,575,0
4,132545,88.0,0.0,-1.0,3.0,-1.0,103.000000,6.000000,97.0,109.0,...,30.00,40.00,4.300000,0.500000,3.8,4.8,4.05,4.55,918,0


In [24]:
#Пропущенных значений не осталось, посмотрим на данные
df.describe()

,RecordID,Age,Gender,Height,ICUType,Weight,Platelets_mean,Platelets_std,Platelets_min,Platelets_max,...,BUNquan25,BUNquan75,WBC_mean,WBC_std,WBC_min,WBC_max,WBCquan25,WBCquan75,Survival,In-hospital_death
count,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,...,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000
mean,142672.608375,64.429750,0.561375,88.118225,2.783750,74.533511,204.157717,21.945574,178.181703,232.605845,...,23.646047,27.285279,12.797013,2.481520,10.348569,16.642623,11.392125,13.631409,130.426875,0.140250
std,5857.052574,17.352628,0.498261,86.204173,1.013096,32.406832,103.131323,19.568059,98.370386,114.377794,...,19.388313,21.542482,21.254923,48.922688,5.907541,140.027899,6.483226,8.431319,364.766144,0.347268
min,132539.000000,15.000000,-1.000000,-1.000000,1.000000,-1.000000,12.166667,0.000000,5.000000,14.000000,...,1.000000,1.500000,0.075000,0.000000,0.000000,0.100000,0.075000,0.100000,-23.000000,0.000000
25%,137592.750000,53.000000,0.000000,-1.000000,2.000000,62.000000,137.000000,9.426985,113.000000,160.000000,...,11.750000,14.000000,8.800000,0.718940,7.200000,10.200000,7.950000,9.450000,-1.000000,0.000000
50%,142674.000000,67.000000,1.000000,152.400000,3.000000,76.000000,188.666667,17.500000,163.000000,213.000000,...,17.500000,20.500000,11.600000,1.473728,9.600000,13.700000,10.550000,12.500000,-1.000000,0.000000
75%,147737.500000,78.000000,1.000000,170.200000,4.000000,90.100000,247.000000,29.000000,222.000000,279.000000,...,28.000000,32.500000,14.868750,2.524288,12.500000,17.900000,13.600000,16.231250,17.000000,0.000000
max,152864.000000,90.000000,1.000000,462.300000,4.000000,472.000000,1483.666667,535.466982,918.000000,2292.000000,...,174.750000,200.000000,1791.542857,4371.709928,201.600000,12500.000000,247.725000,367.050000,2620.000000,1.000000


In [25]:
def replace_values_by_mean(df, column, value1):
    sum_ = 0
    count = 0
    array = df[column].values
    
    for i in array:
        if( i != value1):
            count += 1
            sum_ += i 

    if(count != 0):
        mean_ = sum_/count

    for i in range(0,  array.shape[0]):
        if(array[i] == value1):
            array[i] = mean_
    
    df[column] = array 

In [26]:
replace_values_by_mean(df, 'Weight', -1)
replace_values_by_mean(df, 'Height', -1)

df = df.drop(df[df['Gender'] == -1].index)# Значений, где не указан пол очень мало, их можно выкинуть
df = df.drop(['RecordID'], axis = 1)#Значение не несёт в себе полезной информации

df.Gender = df.Gender.astype(int)
df.ICUType = df.ICUType.astype(int)
df.Age = df.Age.astype(int)

In [27]:
Q1 = df.quantile(0.05)
Q3 = df.quantile(0.95)
IQR = Q3 - Q1
df = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]

In [28]:
df.describe()

,Age,Gender,Height,ICUType,Weight,Platelets_mean,Platelets_std,Platelets_min,Platelets_max,Plateletsquan25,...,BUNquan25,BUNquan75,WBC_mean,WBC_std,WBC_min,WBC_max,WBCquan25,WBCquan75,Survival,In-hospital_death
count,7185.000000,7185.000000,7185.000000,7185.000000,7185.000000,7185.000000,7185.000000,7185.000000,7185.000000,7185.000000,...,7185.000000,7185.000000,7185.000000,7185.000000,7185.000000,7185.000000,7185.000000,7185.000000,7185.000000,7185.000000
mean,64.741545,0.559638,169.655477,2.783020,81.111512,202.784296,21.567778,177.222659,230.652986,189.722538,...,22.650829,25.987398,12.327621,1.851927,10.199666,14.681048,11.208564,13.345169,128.726096,0.138065
std,17.305783,0.496465,7.850862,1.021261,21.541347,97.849414,17.415917,94.507894,106.223650,95.671426,...,17.617919,19.244005,5.363660,1.575432,4.700263,6.665372,4.994156,5.910607,360.174710,0.344993
min,15.000000,0.000000,121.900000,1.000000,21.700000,12.166667,0.000000,5.000000,14.000000,6.750000,...,1.000000,1.500000,0.075000,0.000000,0.000000,0.100000,0.075000,0.100000,-23.000000,0.000000
25%,54.000000,0.000000,169.724569,2.000000,67.100000,137.333333,9.500000,113.000000,160.000000,125.250000,...,11.750000,13.750000,8.833333,0.722842,7.200000,10.300000,8.000000,9.450000,-1.000000,0.000000
50%,68.000000,1.000000,169.724569,3.000000,80.000000,188.333333,17.500000,163.000000,213.000000,175.000000,...,17.250000,20.000000,11.600000,1.471960,9.600000,13.700000,10.550000,12.500000,-1.000000,0.000000
75%,78.000000,1.000000,170.200000,4.000000,90.000000,245.333333,28.662790,220.000000,278.000000,231.500000,...,27.000000,30.750000,14.825000,2.500000,12.500000,17.800000,13.550000,16.150000,16.000000,0.000000
max,90.000000,1.000000,208.300000,4.000000,220.000000,837.000000,136.103719,786.000000,936.000000,811.500000,...,141.000000,159.000000,46.666667,12.514259,41.400000,57.300000,41.400000,51.150000,2408.000000,1.000000


In [29]:
df

,Age,Gender,Height,ICUType,Weight,Platelets_mean,Platelets_std,Platelets_min,Platelets_max,Plateletsquan25,...,BUNquan25,BUNquan75,WBC_mean,WBC_std,WBC_min,WBC_max,WBCquan25,WBCquan75,Survival,In-hospital_death
0,54,0,169.724569,4,81.471419,203.000000,18.000000,185.0,221.0,194.00,...,9.25,11.75,10.300000,0.900000,9.4,11.2,9.850,10.750,-1,0
1,76,1,175.300000,2,76.000000,178.600000,29.789931,135.0,226.0,164.00,...,17.00,19.50,11.266667,2.735365,7.4,13.3,10.250,13.200,-1,0
4,88,0,169.724569,3,81.471419,103.000000,6.000000,97.0,109.0,100.00,...,30.00,40.00,4.300000,0.500000,3.8,4.8,4.050,4.550,918,0
5,64,1,180.300000,1,114.000000,210.750000,41.918820,173.0,276.0,174.50,...,15.75,17.50,16.100000,4.601630,12.7,24.0,13.150,16.800,1637,0
6,68,0,162.600000,3,87.000000,329.666667,5.249339,325.0,337.0,326.00,...,31.00,33.00,6.366667,0.235702,6.2,6.7,6.200,6.450,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7994,64,0,169.724569,4,70.000000,112.500000,9.500000,103.0,122.0,107.75,...,22.50,23.50,14.250000,3.050000,11.2,17.3,12.725,15.775,2210,0
7995,78,1,180.300000,2,83.180000,118.250000,9.496710,103.0,128.0,114.25,...,23.50,25.00,16.766667,2.347102,14.5,20.0,15.150,17.900,752,0
7996,90,1,177.800000,3,87.000000,278.428571,60.674441,191.0,351.0,227.50,...,51.00,57.50,28.071429,8.713489,18.0,41.8,20.450,34.200,39,0
7997,70,0,152.400000,2,74.000000,189.333333,27.980152,160.0,227.0,170.50,...,28.50,30.50,14.333333,0.880656,13.1,15.1,13.950,14.950,334,0


In [30]:
df = pd.get_dummies(df, columns=['ICUType'])#Это следует сделать в последнюю очередь, чтобы можно было сделать визуализацию 
df.to_csv('classification_dataset.csv', index=False)